In [3]:
import json
import pandas as pd
import re
import os
from copy import copy
import math


from transformers import AutoTokenizer, AutoModelWithLMHead, get_scheduler
import torch
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

In [ ]:
DATA_DIR = '../data'
df = pd.read_csv(os.path.join(DATA_DIR, 'kremlin_ner_dataset.csv'))

In [ ]:
Больше половины текстов имеет тэг "Поздравления", оставлю только их:

In [ ]:
df['tags'].value_counts()

In [31]:
# Есть тексты без тэга, но их мало. Проверю их тематику вручную
df['tags'].isna().mean()

0.006731173748422381

Некоторые тексты без тэгов - поздравления. Это тексты с индексами 851, 1227, 1374 и 2261

In [23]:
df[df['tags'].isna()]['title']

69      Читателям научно-популярного комментария к Кон...
406     Реджепу Тайипу Эрдогану, Президенту Турецкой Р...
431     Галине Волчек, художественному руководителю Мо...
442     Участникам и гостям хоккейного матча, приуроче...
542     Участникам, организаторам и гостям чемпионата ...
851     Юрию Григоровичу, хореографу, народному артист...
1137     Борису Джонсону, Премьер-министру Великобритании
1227    Победителю чемпионата мира по тяжёлой атлетике...
1374    Александру Ширвиндту, художественному руководи...
1677    Ги Пармелену, Президенту Швейцарской Конфедерации
1802    Участникам и гостям мемориального вечера «Хран...
1910    Участникам XVII совещания руководителей специа...
1992    Участникам торжественной церемонии открытия му...
2050                 Жаиру Болсонаро, Президенту Бразилии
2205    В.Г.Береговому, Л.Г.Елисеевой (Береговой), М.Т...
2261    Наталье Авдеевой, победительнице чемпионата ми...
Name: title, dtype: object

In [32]:
extra_congratulation_df = df.loc[[851, 1227, 1374, 2261]]

In [33]:
df = df.dropna()
df = df[df['tags'].apply(lambda x: 'Поздравления' in x)]

In [34]:
df = pd.concat([df, extra_congratulation_df])

Посмотрим, есть ли дубли. При скачивании проверялось, что нет случаев полного совпадения ссылки, но один и тот же текст мог быть опубликован с разными ссылками

In [58]:
len(df)

1262

In [60]:
len(set(df['title']))

1104

Есть совпадающие заголовки, посмотрим на эти случаи

In [61]:
duplicates = []

for title, group in df.groupby('title'):
    if len(group) > 1:
        duplicates.append(list(group['text']))

In [63]:
duplicates[0]

['Уважаемая Александра Николаевна!\nПримите сердечные поздравления с Днём рождения.\nВы принадлежите к легендарной плеяде мастеров, в творчестве которых нашли отражение наиболее яркие, незабываемые страницы отечественной истории, победы и свершения многих поколений, судьба страны.\nЗамечательные песни, все музыкальные произведения, созданные Вами за годы вдохновенного труда, по праву являются нашим национальным достоянием.\nЖелаю Вам здоровья, благополучия и всего самого доброго.',
 'Уважаемая Александра Николаевна!\nПримите поздравления с Днём рождения.\nВас по праву считают выдающимся представителем отечественной культуры, бесконечно талантливым и доброжелательным человеком, великим тружеником и созидателем.\nУбеждён, что главный секрет Вашего творческого долголетия – искренняя увлечённость любимым делом и огромное уважение к людям, которым Вы щедро дарите тепло своей души и сердца.\nПозвольте, дорогая Александра Николаевна, пожелать Вам и Николаю Николаевичу здоровья, благополучия и

In [65]:
duplicates[1]

['Уважаемый Александр Александрович!\nПримите искренние поздравления с 75-летним юбилеем.\nЩедрый талант и вдохновенный творческий труд в кинематографе и на театральной сцене принесли Вам всенародную любовь и профессиональное признание.\nОтрадно, что и сегодня Вы работаете с полной душевной отдачей, своей многогранной деятельностью вносите значимый вклад в сохранение замечательных традиций отечественного искусства.\nОт души желаю Вам доброго здоровья и всего наилучшего.',
 'Уважаемый Александр Александрович!\nПримите поздравления с 80-летним юбилеем.\nБольшой и щедрый талант, безграничная преданность избранному делу помогли Вам добиться признания на ниве созидания и творчества, воплотить в театре и кинематографе яркие, неординарные идеи и замыслы. Важно, что и сегодня Вы объединяете вокруг себя замечательный коллектив Московского театра Et Cetera, радуете публику интересными спектаклями и громкими премьерами. И конечно, отмечу Вашу плодотворную, востребованную работу на посту руководит

In [66]:
duplicates[2]

['Уважаемый Александр Григорьевич,\nпримите самые искренние поздравления по случаю Дня рождения.\nЗа годы работы на посту главы государства Вы снискали высокое уважение соотечественников и значительный авторитет за рубежом как ответственный и дальновидный руководитель, многое делающий для социально-экономического развития Беларуси и защиты её внешнеполитических интересов.\nВесьма велик Ваш личный вклад в укрепление дружественных отношений между нашими странами, в ускорение взаимовыгодных интеграционных процессов в рамках Союзного государства и Евразийского экономического союза Краткая справка Евразийский экономический союз (ЕАЭС) .\nХотел бы подтвердить готовность к продолжению нашего товарищеского диалога и конструктивной совместной работы по актуальным вопросам двусторонней, региональной и международной повестки дня. Уверен, это отвечает коренным интересам братских народов России и Беларуси.\nОт души желаю Вам, уважаемый Александр Григорьевич, крепкого здоровья, счастья, благополучия

Совпадающие заголовки - поздравления с днем рождения одного и того же человека в разные годы

In [36]:
for column in ['title_ner', 'text_ner']:
    df[column] = df[column].apply(json.loads)

In [4]:
SELECTED_ENTITIES = ['PER', 'ORG']

BOS_TAG = 'BOS'
EOS_TAG = 'EOS'
TITLE_TAG = 'TITLE'
TEXT_TAG = 'TEXT'

In [38]:
def join_entities(title_ner, text_ner, selected_entities=SELECTED_ENTITIES):
    '''
    Для выбранных типов сущностей берет значение из сущностей заголовка (при наличии).
    Если сущностей нет в заголовке, берет из текста
    '''
    joined_entities = {}
    for entity_type in selected_entities:
        for entities in [title_ner, text_ner]:
            if entity_type in entities:
                joined_entities[entity_type] = entities[entity_type]
                break
    return joined_entities

In [39]:
df['selected_entities'] = df.apply(
    lambda row: join_entities(row['title_ner'], row['text_ner']),
    axis=1
)

In [40]:
df['selected_entities'].apply(len).describe()

count    1299.000000
mean        1.063895
std         0.568631
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: selected_entities, dtype: float64

Сущности не выделены меньше, чем для четверти примеров

In [41]:
def prepare_text_for_gpt(text:str):
    '''
    Функция добавляет пробелы после разделителей строки и точек, если пробелов там не было,
    потому что при токенизации пробел является частью первого токена слова.
    Эмбединги для случаев "заглавная буква идет не после пробела" предобучены хуже.
    '''
    text = re.sub('\n', ' ', text)
    text = re.sub(re.escape('.'), '. ', text)
    # если были добавлены двойные пробелы, убираем их
    text = re.sub(' +', ' ', text)
    return text

def format_prompt(row:pd.core.series.Series):
    prompt = ''
    for entity_tag in SELECTED_ENTITIES:
        entity_content = ', '.join(row['selected_entities'].get(entity_tag, []))
        prompt += f'{entity_tag} {entity_content} '
    prompt = f"{BOS_TAG} {prompt}{TITLE_TAG} {row['title']} {TEXT_TAG} {row['text']} {EOS_TAG}"
    return prepare_text_for_gpt(prompt)

In [42]:
df['prompt'] = df.apply(format_prompt, axis=1)

Посмотрим распределение длин токенизированных текстов

In [5]:
PRETRAINED_MODEL_NAME = 'sberbank-ai/rugpt3small_based_on_gpt2'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tokenizer.add_special_tokens({'bos_token':BOS_TAG,
                              'eos_token':EOS_TAG,
                              'pad_token': '[PAD]',
                              'additional_special_tokens':[TITLE_TAG, TEXT_TAG, *SELECTED_ENTITIES]})

7

In [46]:
tokenized = []
prompts = list(df['prompt'])
encoded_prompts_lens = []
tokenization_batch_size = 100

for batch_idx in range(math.ceil(len(prompts)/tokenization_batch_size)):
    batch_start = batch_idx * tokenization_batch_size
    batch_end = (batch_idx+1) * tokenization_batch_size
    batch = prompts[batch_start:batch_end]
    # при большом объеме данных стоило бы сохранить результат токенизации и один раз допадить, 
    # но сейчас токенизация не занимает много времени
    encoded_prompts_lens += [len(i) for i in tokenizer(batch)['input_ids']]
df['encoded_prompts_lens'] = encoded_prompts_lens

In [47]:
df['encoded_prompts_lens'].describe()

count    1299.000000
mean      178.998460
std        56.873538
min        82.000000
25%       130.000000
50%       172.000000
75%       216.000000
max       480.000000
Name: encoded_prompts_lens, dtype: float64

Тексты не превышают ограничения языковой модели, но я все равно уберу самые длинные, чтобы избежать превышения памяти cuda

In [48]:
length_treshold = 300
df = df[df['encoded_prompts_lens']<=length_treshold]

In [49]:
df['encoded_prompts_lens'].describe()

count    1262.000000
mean      174.419176
std        50.583849
min        82.000000
25%       129.000000
50%       170.000000
75%       212.000000
max       300.000000
Name: encoded_prompts_lens, dtype: float64

In [50]:
len(df)

1262

In [51]:
train, test = train_test_split(df, test_size=0.1, random_state=42)

Отметим для тестовых примеров, были ли такие же заголовки представлены в трейне

In [67]:
known_titles = set(train['title'])
test['title_is_known'] = test['title'].apply(lambda x: x in known_titles)

In [56]:
for column in ['title_ner', 'text_ner', 'selected_entities']:
     test[column] = test[column].apply(json.dumps)

In [68]:
test.to_csv(os.path.join(DATA_DIR, 'test_prompts.csv'), index=False)

In [9]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [10]:
train = pd.read_csv('gdrive/MyDrive/train_prompts.csv')

In [25]:
test = pd.read_csv('gdrive/MyDrive/test_prompts.csv')

In [16]:
# для ускорения обучения полезно сделать так, чтобы в батче подавались тексты примерно одной длины (будет меньше падингов)
# ascending=False, чтобы в случае превышения памяти на самых длинных текстах сразу увидеть эту проблему
# стоило бы сделать так, чтобы короткие и длинные батчи подавались в перемешку, но (зачеркнуто: мне лень) 
# при небольшом количестве итераций в эпохе вряд ли модель успеет переобучиться на длину последних (коротких) текстов
train.sort_values('encoded_prompts_lens', ascending=False, inplace=True)

In [12]:
train_batch_size = 5
train_dataloader = torch.utils.data.DataLoader(list(train['prompt']), batch_size=train_batch_size)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
num_epochs = 10
num_training_steps = num_epochs*len(train_dataloader)

In [15]:
gpt = AutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL_NAME).to(device).train()
gpt.resize_token_embeddings(len(tokenizer))
optimizer = torch.optim.AdamW(gpt.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]


В генеративных задачах модель с лучшими метриками не обязательно выдает лучший с точки зрения соответствия целевой задаче результат.
Для ускорения работы сначала дообучу модель, а потом уже посчитаю метрики на тесте без подсчета метрик для чекпойнтов.

In [16]:
SAVE_PATH = 'gdrive/MyDrive/gpt_small_president_letter'

In [17]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        tokenized_batch = tokenizer(list(batch), return_tensors='pt', padding=True).to(device)
        outputs = gpt(**tokenized_batch, labels=tokenized_batch['input_ids'])
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    gpt.save_pretrained(SAVE_PATH)

  0%|          | 0/2270 [00:00<?, ?it/s]

#Проверка на адекватность результата и подбор тактики получения повтора

Более детальный анализ и подбор параметров будет в другой тетрадке

In [ ]:
gpt.eval()

In [23]:
test.reset_index(inplace=True)

In [24]:
real_prompt = test['prompt'].loc[0]

In [25]:
real_prompt

'BOS PER ORG Геральдический совет, Геральдический совет TITLE Участникам конференции, посвящённой 300-летию учреждения и 30-летию восстановления государственной геральдической службы TEXT Уважаемые друзья! Приветствую вас по случаю открытия конференции и поздравляю с 300-летием учреждения и 30-летием восстановления государственной геральдической службы. Отрадно, что 30 лет назад для укрепления исторической связи времён и поколений, формирования и реализации единой государственной политики в сфере геральдики был образован Геральдический совет при Президенте Российской Федерации. За прошедшие годы он внёс значимый вклад в создание соответствующей нормативной базы, в становление важнейших символов российской государственности и практики их использования. И конечно, отмечу творческую, кропотливую деятельность совета, направленную на развитие и совершенствование региональной символики нашей многонациональной страны, в которой в полной мере отразились богатейшие ратные и созидательные традиц

In [26]:
prompt_meta = real_prompt.split(TITLE_TAG)[0] + TITLE_TAG
prompt_meta_with_beginning = real_prompt.split(TEXT_TAG)[0] + TEXT_TAG

In [27]:
prompt_meta

'BOS PER ORG Геральдический совет, Геральдический совет TITLE'

In [28]:
prompt_meta_with_beginning

'BOS PER ORG Геральдический совет, Геральдический совет TITLE Участникам конференции, посвящённой 300-летию учреждения и 30-летию восстановления государственной геральдической службы TEXT'

In [20]:
def generate(prompt):
   encoding = tokenizer([prompt], return_tensors='pt').to(device)
   output = gpt.generate(**encoding, 
                         min_length=50,
                         max_length=250,
                         bad_words_ids=[[tokenizer.pad_token_id]],
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id=tokenizer.pad_token_id)
   return tokenizer.decode(output[0])

In [30]:
generate(prompt_meta)

'BOS PER ORG Геральдический совет, Геральдический совет TITLE Участникам торжественного заседания, посвящённого 100-летию учреждения Геральдического совета при Министерстве Российской Федерации по делам гражданской обороны, чрезвычайным ситуациям и ликвидации последствий стихийных бедствий TEXT Уважаемые друзья! Поздравляю вас с 100-летием учреждения Геральдического совета при Министерстве Российской Федерации по делам гражданской обороны, чрезвычайным ситуациям и ликвидации последствий стихийных бедствий. За прошедшие годы Совет вырос в авторитетный общественный, экспертный центр, внёс значимый вклад в развитие отечественной системы государственного управления. В его деятельности неизменно принимают участие известные учёные, видные государственные и общественные деятели, видные общественные и государственные деятели. Важно, что вы достойно продолжаете традиции своих предшественников, активно внедряете новаторские идеи и подходы, активно взаимодействуете с федеральными и региональными 

In [31]:
generate(prompt_meta_with_beginning)

'BOS PER ORG Геральдический совет, Геральдический совет TITLE Участникам конференции, посвящённой 300-летию учреждения и 30-летию восстановления государственной геральдической службы TEXT Дорогие друзья! Поздравляю вас со знаменательной датой – 300-летием учреждения и 30-летия восстановления государственной геральдической службы России. Этот юбилей – важное событие для отечественной культуры, для укрепления отечественного исторического, культурного, духовного наследия, для сбережения нашего богатейшего исторического, духовного наследия. Мы по праву гордимся многими поколениями своих предшественников, которые внесли значимый вклад в развитие отечественной культуры, в укрепление российской государственности, в развитие музейного дела, краеведения, других творческих, просветительских инициатив. Важно, что вы достойно продолжаете традиции своих предшественников, активно участвуете в реализации востребованных образовательных, просветительских, патриотических проектов, которые получают широк

In [32]:
real_prompt = test['prompt'].loc[1]
prompt_meta = real_prompt.split(TITLE_TAG)[0] + TITLE_TAG
prompt_meta_with_beginning = real_prompt.split(TEXT_TAG)[0] + TEXT_TAG

In [33]:
generate(prompt_meta)

'BOS PER ORG Серовского исторического музея TITLE Коллективу Серовского исторического музея TEXT Дорогие друзья! Поздравляю вас со 100-летием Серовского исторического музея. Основанный в 1918 году, музей стал одним из крупнейших в России музеев истории родного края и сегодня по праву считается одним из крупнейших культурных, просветительских центров. Здесь представлены произведения древнерусской живописи, декоративно-прикладного искусства, археологические находки, рассказывающие посетителям об истории Серовского края, его замечательных традициях и обычаях, о самобытных традициях и обычаях проживающих здесь народов. Важно, что нынешний коллектив бережно хранит и пополняет экспозицию музея, уделяет неустанное внимание просветительской, познавательной деятельности, уделяет неустанное внимание реализации востребованных образовательных, информационных, творческих проектов. Желаю вам успехов и всего наилучшего. EOS'

In [34]:
generate(prompt_meta_with_beginning)

'BOS PER ORG Серовского исторического музея TITLE Коллективу Серовского исторического музея TEXT Дорогие друзья! Поздравляю вас со 100-летием Серовского исторического музея. Основанный в 1918 году, музей стал одним из крупнейших в России музеев истории родного края и сегодня по праву считается одним из крупнейших культурных, просветительских центров. Здесь представлены произведения древнерусской живописи, декоративно-прикладного искусства, археологические находки, рассказывающие посетителям об истории Серовского края, его замечательных традициях и обычаях, о самобытных традициях и обычаях проживающих здесь народов. Важно, что нынешний коллектив бережно хранит и пополняет экспозицию музея, уделяет неустанное внимание просветительской, познавательной деятельности, уделяет неустанное внимание реализации востребованных образовательных, информационных, творческих проектов. Желаю вам успехов и всего наилучшего. EOS'

Модель справляется с повторением указанных сущностей-организаций. Проверим, что будет с персонами

In [44]:
generate("BOS PER Анна Ахматова ORG TITLE")

'BOS PER Анна Ахматова ORG TITLE Анне Ахматовой, поэту, лауреату Государственной премии СССР TEXT Уважаемая Анна Андреевна! Примите поздравления с юбилейным Днём рождения. Вас по праву считают одним из наиболее талантливых, самобытных поэтов современности. Многие годы Вы преданно служите избранному делу, своим творчеством обогащаете замечательные традиции русской поэзии. Отрадно, что и сегодня Вы продолжаете радовать поклонников новыми замечательными стихотворениями, радуете публику интересными, содержательными творческими находками. Желаю Вам здоровья, вдохновения и всего наилучшего. EOS'

In [45]:
generate("BOS PER Марк Розовский ORG TITLE")

'BOS PER Марк Розовский ORG TITLE Марку Розовскому, художнику-модельеру, педагогу, народному художнику России TEXT Уважаемый Марк Анатольевич! Примите поздравления с 80-летним юбилеем. Талантливый художник, скульптор, автор уникальных работ, созданных в самых разных жанрах и техниках, Вы добились впечатляющих профессиональных успехов, создали замечательные иллюстрации к детским книгам, которые учат дружбе, взаимопомощи, любви к родному краю. И конечно, отмечу Вашу многогранную просветительскую, наставническую деятельность, искреннюю заботу о сохранении и развитии замечательных традиций отечественного искусства. Желаю Вам здоровья, вдохновения и всего самого доброго. EOS'

In [47]:
generate("BOS PER Анна Щербакова ORG TITLE")

'BOS PER Анна Щербакова ORG TITLE Анне Щербаковой, победительнице XXIX Всемирной зимней универсиады 2019 года в Красноярске в соревнованиях по сноуборду в слоуп-стайле TEXT Уважаемая Анна Сергеевна! Поздравляю Вас с «золотом» красноярской универсиады. Вы достойно представили нашу страну на «домашней» универсиаде, продемонстрировали командную сплочённость и нацеленность на победу. И конечно, особые слова благодарности – тренерам, наставникам, специалистам, всем, кто помогает Вам добиваться успеха. Желаю новых достижений и всего наилучшего. EOS'

С общеизвестными персонами модель справляется и поомнит примерную сферу деятельности (Ахматова ожила и поменяла свое отношение к СССР, Розовский стал модельером, а Щербакова - сноубордисткой)

In [41]:
generate("BOS PER В. С. Иванов ORG TITLE")

'BOS PER В. С. Иванов ORG TITLE Работникам и ветеранам судостроительной отрасли России TEXT Уважаемые друзья! Поздравляю вас с Днём кораблестроителя. Этот праздник отмечают инженеры, конструкторы, инженеры, представители рабочих профессий, все, кто посвятил себя развитию отечественной экономики, оборонно-промышленного комплекса, укреплению обороноспособности страны. Отрадно, что сегодня отрасль уверенно развивается, наращивает свой потенциал, уверенно наращивает темпы жилищного строительства, реализует востребованные проекты в сфере промышленности и АПК, вносит серьёзный вклад в укрепление национальной экономики и социальной сферы. Уверен, что вы и впредь будете добросовестно трудиться, достойно решать поставленные задачи на благо России и её граждан. Желаю вам успехов и всего наилучшего. EOS'

In [43]:
generate("BOS PER А. Петров ORG TITLE")

'BOS PER А. Петров ORG TITLE Участникам торжественного мероприятия, посвящённого 100-летию создания в России системы скорой медицинской помощи TEXT Уважаемые друзья! Поздравляю вас с юбилеем создания в России системы скорой медицинской помощи. За прошедшие годы она доказала свою эффективность, стала одним из ключевых, ключевых факторов социально-экономического развития страны, стала надёжным, социально значимым звеном отечественной скорой медицинской помощи. Сегодня перед вами стоят серьёзные, востребованные временем задачи, главные из которых – создание современных, современных, высокотехнологичных медицинских услуг, совершенствование профильного здравоохранения, совершенствование диагностики и лечения заболеваний. И конечно, необходимо уделять приоритетное внимание вопросам экологии, сбережения нашего бесценного природного богатства. Желаю вам успехов и всего самого доброго. EOS'

С обычными фамилиями модель справляется хуже

In [49]:
generate("BOS PER В. С. Иванову ORG TITLE")

'BOS PER В. С. Иванову ORG TITLE В. С. Иванову, победителю XXIX Всемирной зимней универсиады 2019 года в Красноярске в соревнованиях по сноуборду в слоуп-стайле TEXT Уважаемый Владимир Сергеевич! Поздравляю Вас с «золотом» красноярской универсиады. Вы достойно представили нашу страну на «домашней» универсиаде, продемонстрировали отличную подготовку и по-настоящему командный дух. Завоёванное Вами «золото» – заслуженная награда за упорный труд, настойчивость и целеустремлённость. Желаю Вам новых успехов и всего самого доброго. EOS'

In [50]:
generate("BOS PER В. С. Петрову ORG TITLE")

'BOS PER В. С. Петрову ORG TITLE В. С. Петрову, победителю XXIX Всемирной зимней универсиады 2019 года в Красноярске в соревнованиях по сноуборду в слоуп-стайле TEXT Уважаемый Владимир Сергеевич! Поздравляю Вас с триумфом. На соревнованиях в Красноярске Вы проявили себя как сильный, волевой, волевой, нацеленный на результат спортсмен, достойный представитель отечественной школы сноубординга. В напряжённой борьбе с опытными соперниками проявили недюжинный спортивный характер, мастерство и по-настоящему бойцовский характер. И конечно, Ваш успех – значимый вклад в развитие замечательных традиций отечественной школы сноубординга. Желаю Вам новых достижений и всего самого доброго. EOS'

Но ситуация поправляется, если ставить фамилию в дательный падеж

Текст генерируется примерно одинаковый. Возможно, nucleus sampling справится с задачей лучше. Проверю это при подборе параметров

Проверим фамилии другого типа

In [52]:
generate("BOS PER Лотману ORG TITLE")

'BOS PER Лотману ORG TITLE Лотману, художнику-модельеру, народному художнику СССР TEXT Уважаемый Лотман! Примите поздравления с 80-летним юбилеем. Вас знают и ценят как талантливого художника, скульптора, как человека, искренне любящего и знающего своё дело. Ваши работы всегда интересны и самобытны, они всегда становятся настоящим событием в мире моды и дизайна, украшают коллекции знаменитых мировых дизайнеров и других известных людей. Желаю Вам здоровья, вдохновения и всего наилучшего. EOS'

In [53]:
generate("BOS PER Бонч-Осмаловской ORG TITLE")

'BOS PER Бонч-Осмаловской ORG TITLE Участникам торжественного мероприятия, посвящённого 100-летию создания в России системы скорой медицинской помощи TEXT Уважаемые друзья! Поздравляю вас с 100-летием создания в России системы скорой медицинской помощи. За прошедшие годы она доказала свою эффективность, стала одним из ключевых, ключевых факторов развития высокотехнологичной медицины, стала настоящим центром притяжения для людей, оказавшихся в сложной жизненной ситуации. Важно, что сегодня вы бережно храните и развиваете замечательные традиции милосердия и подвижничества, заложенные предшественниками, стремитесь эффективно решать стоящие перед медициной задачи, главные из которых – создание современных, современных, современных медицинских услуг, совершенствование диагностической, лечебной базы, совершенствование технологий и методов лечения. И конечно, важно, что вы активно участвуете в реализации востребованных благотворительных, образовательных, просветительских инициатив, активно вз

In [54]:
generate("BOS PER Бонч-Осмаловской ORG TITLE Бонч-Осмаловской ")

'BOS PER Бонч-Осмаловской ORG TITLE Бонч-Осмаловской TEXT Дорогие друзья! Поздравляю вас с 75-летием освобождения Севастополя от немецко-фашистских захватчиков. Севастополь всегда играл особую роль в истории нашей страны, в укреплении российской государственности. Он всегда был надёжной крепостью, обороной Севастополя и Крыма. В памяти народа навеки останется подвиг солдат и офицеров Красной армии, моряков и ополченцев, которые в январе 1944 года в ходе упорных боёв освободили город и весь Крым от немецко-фашистских захватчиков. Севастопольцы свято чтят подвиги отцов и дедов, свято чтят подвиги солдат и офицеров Красной армии, которые в мае 1944 года в ходе упорных боёв освободили полуостров Крым от немецко-фашистских захватчиков. Севастопольцы чтят подвиг солдат и офицеров Красной армии, которые в мае 1944 года в ходе упорных боёв освободили полуостров Крым от немецко-фашистских захватчиков. И конечно, Севастополь гордится героическими страницами ратной летописи Отечества, именами отц

С некоторыми фамилиями не справляется, даже если передать их в после TITLE

In [55]:
generate("BOS PER Бонч-Осмаловской ORG TITLE А. А. Бонч-Осмаловской ")

'BOS PER Бонч-Осмаловской ORG TITLE А. А. Бонч-Осмаловской TEXT Уважаемая Александра Александровна! Примите поздравления с Днём рождения. Вы посвятили себя служению искусству, добились выдающихся профессиональных успехов, внесли значимый вклад в развитие отечественной культуры. Ваши работы в кинематографе и на театральной сцене – свидетельство большого, искреннего уважения к людям, которые посвятили себя служению искусству. Желаю Вам здоровья, благополучия и всего самого доброго. EOS'

In [59]:
generate("BOS PER Бонч-Осмаловской ORG TITLE Г. Н. Бонч-Осмаловской")

'BOS PER Бонч-Осмаловской ORG TITLE Г. Н. Бонч-Осмаловской, лётчику-космонавту, Герою Советского Союза TEXT Уважаемая Галина Николаевна! Примите поздравления с Днём рождения. Вы посвятили свою жизнь сбережению памяти о Великой Отечественной войне, внесли значимый личный вклад в развитие отечественной космонавтики, в укрепление обороноспособности и национальной безопасности страны. И конечно, глубокого уважения заслуживает Ваша ответственная гражданская позиция, значимый вклад в развитие межпланетных и суборбитальных связей. Желаю Вам здоровья, благополучия и всего самого доброго. EOS'

Помогает подставить какие-то инициалы




Проверим, как модель справляется с организациями, не похожими на трейн

In [60]:
generate("BOS PER ORG Психиатрическая больница №1 TITLE")

'BOS PER ORG Психиатрическая больница №1 TITLE Сотрудникам и ветеранам медицинского учреждения, сотрудникам и ветеранам медицинского учреждения «Психиатрическая больница №1» TEXT Уважаемые друзья! Поздравляю вас со знаменательной датой – 100-летием медицинского учреждения «Психиатрическая больница №1». История одного из старейших в России лечебных учреждений поистине впечатляет. Её история берёт начало в 1918 году, когда по инициативе врачей-психиатров, врачей-психиатров, медицинских сестёр милосердия и сестёр милосердия была создана первая в мире стационарная больница. За прошедшее время она прошла большой, насыщенный путь, стала одним из признанных лидеров отечественного здравоохранения. Сегодня в стенах больницы трудятся специалисты высочайшей квалификации, которые искренне заботятся о здоровье людей, ведут обширную просветительскую, благотворительную деятельность, многое делают для решения актуальных проблем современности. Важно, что вы с уважением относитесь к традициям милосердия

In [61]:
generate("BOS PER ORG Фонд помощи ЛГБТ TITLE")

'BOS PER ORG Фонд помощи ЛГБТ TITLE Участникам и гостям торжественного мероприятия, посвящённого 10-летию Фонда «Против ЛГБТ» TEXT Дорогие друзья! Поздравляю вас с 10-летием Фонда «Против ЛГБТ». За прошедшие годы Фонд вырос в крупную, авторитетную общественную организацию, многое сделал для продвижения ценностей и ценностей, лежащих в основе современного общества, укрепления института семьи, воспитания подрастающего поколения. И конечно, особо отмечу вашу большую, востребованную работу по оказанию действенной, действенной помощи тем, кто в наше сложное время нуждается в помощи и поддержке. Важно, что вы с уважением относитесь к традициям, заложенным ветеранами, стремитесь эффективно решать стоящие перед Фондом задачи, главные из которых – поддержка наиболее значимых групп населения, создание условий для их активного участия в жизни страны, гармонизация межнациональных и межрелигиозных отношений, создание условий для развития и популяризации ЛГБТ-культа. И конечно, важно, что Фонд уделя

In [68]:
generate("BOS PER ORG Европейский суд по правам человека TITLE")

'BOS PER ORG Европейский суд по правам человека TITLE Судьям Конституционного Суда TEXT Уважаемые судьи Конституционного Суда Российской Федерации! Уважаемые коллеги! Поздравляю вас с 30-летием создания Конституционного Суда Российской Федерации. За прошедшие годы Конституционный Суд прошёл большой путь, накопил серьёзный опыт в обеспечении единства правового пространства страны, в реализации значимых социальных, инфраструктурных, миротворческих проектов. И сегодня он по праву пользуется высоким авторитетом как в отечественной правовой системе, так и в мировой практике. Конституционный Суд играет важную роль в обеспечении единства правового пространства страны, в обеспечении единства правоприменительной практики, в обеспечении единства и равноправия в обществе. Уверен, что вы и впредь будете добросовестно трудиться, эффективно решать поставленные задачи на благо России и её граждан. Желаю вам успехов и всего наилучшего. EOS'

С генерацией текстов, не соответствующих позиции чиновников, есть предсказуемые проблемы. 

In [62]:
generate("BOS PER ORG Фонд помощи беженцам TITLE")

'BOS PER ORG Фонд помощи беженцам TITLE Участникам и гостям торжественного мероприятия, посвящённого 10-летию Фонда помощи беженцам TEXT Дорогие друзья! Поздравляю вас с 10-летием Фонда помощи беженцам. За прошедшие годы Фонд вырос в крупную, авторитетную структуру, многое сделал для защиты прав и интересов людей, оказавшихся в сложной жизненной ситуации. Оказывал действенную помощь тем, кто попал в беду, и внёс свой вклад в решение гуманитарных проблем, в укрепление мира и стабильности в стране. Важно, что сегодня Фонд динамично развивается, реализует востребованные социальные, просветительские, благотворительные проекты, содействует дальнейшему развитию гуманитарных связей, упрочению взаимопонимания между людьми. Убеждён, что вы и впредь будете добросовестно трудиться, достойно решать поставленные задачи, содействовать упрочению дружбы и взаимопонимания между людьми, укреплению доверия и взаимопонимания между людьми. Желаю вам успехов и всего наилучшего. EOS'

In [63]:
generate("BOS PER ORG Фонд помощи бездомным котятам TITLE")

'BOS PER ORG Фонд помощи бездомным котятам TITLE Ветеранам Фонда помощи бездомным котятам TEXT Дорогие друзья! Поздравляю вас с Днём котят. Этот замечательный праздник отмечают представители одной из старейших и наиболее ценных в мире групп котят – представители редкого, долгоживущего вида. Они по праву считаются гордостью и гордостью отечественного и мирового сообщества, служат символом заботы о здоровье и благополучии людей. И конечно, особо отмечу их подвижнический, истинно подвижнический труд, который позволил им вырасти настоящими героями, стать истинными патриотами и ратниками Отечества. Важно, что Фонд активно участвует в жизни страны, укрепляет взаимодействие с общественными организациями, реализует востребованные благотворительные, просветительские, патриотические проекты. Убеждён, что вы и впредь будете беречь и приумножать замечательные традиции своих питомцев, заботиться о них, подавать пример преданного служения Родине. Желаю вам успехов и всего наилучшего. EOS'

С другими названиями такого типа модель справляется намного лучше, хотя все равно подмешивает "типичное" содержание

Попробую уменьшить количество эпох - все-таки модель переучилась на какие-то патерны (хотя, возможно, так и задумано)

# Уменьшаем количество эпох, чтобы избежать переобучения

In [13]:
num_epochs = 3
num_training_steps = num_epochs*len(train_dataloader)

In [14]:
gpt = AutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL_NAME).to(device).train()
gpt.resize_token_embeddings(len(tokenizer))
optimizer = torch.optim.AdamW(gpt.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [15]:
SAVE_PATH = 'gdrive/MyDrive/gpt_small_president_letter_3_epochs'

In [16]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        tokenized_batch = tokenizer(list(batch), return_tensors='pt', padding=True).to(device)
        outputs = gpt(**tokenized_batch, labels=tokenized_batch['input_ids'])
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    gpt.save_pretrained(SAVE_PATH)

  0%|          | 0/681 [00:00<?, ?it/s]

In [ ]:
gpt.eval()

In [21]:
generate("BOS PER ORG Европейский суд по правам человека TITLE")

'BOS PER ORG Европейский суд по правам человека TITLE Жителям Республики Марий Эл TEXT Уважаемые друзья! Поздравляю вас с Днём Республики Марий Эл. Этот праздник отмечают представители разных народов, представители разных социальных, религиозных, культурных, деловых кругов, представители органов власти, бизнеса, представители общественных организаций. Отрадно, что вы бережно храните традиции дружбы и взаимовыручки, с уважением относитесь к традициям своих предшественников, стремитесь эффективно решать стоящие перед республикой задачи. И конечно, особо отмечу ваш значимый вклад в укрепление международного гуманитарного сотрудничества, в развитие взаимовыгодного двустороннего сотрудничества в различных сферах. Желаю вам успехов и всего наилучшего. EOS'

In [22]:
generate("BOS PER ORG Фонд помощи ЛГБТ TITLE")

'BOS PER ORG Фонд помощи ЛГБТ TITLE Участникам и гостям торжественного мероприятия, посвящённого 100-летию Фонда помощи ЛГБТ TEXT Дорогие друзья! Поздравляю вас с 100-летием Фонда помощи ЛГБТ. За прошедшие годы Фонд прошёл большой, насыщенный путь, стал одним из признанных лидеров в сфере благотворительности и просвещения, содействовал развитию института семьи, укреплению института семьи как института гражданского общества, укреплению института семьи как института гражданского общества в нашей стране. Важно, что вы бережно храните и развиваете замечательные традиции милосердия, милосердия и благотворительности, которые передаются из поколения в поколение. Важно, что вы бережно относитесь к бесценному, уникальному наследию своих предшественников, стремитесь содействовать реализации востребованных благотворительных, просветительских, образовательных проектов, содействовать укреплению института семьи, укреплению института семьи как института гражданского общества. Желаю вам успехов и всег

In [23]:
generate("BOS PER ORG Психиатрическая больница №1 TITLE")

'BOS PER ORG Психиатрическая больница №1 TITLE Коллективу и ветеранам Психиатрической больницы №1 TEXT Уважаемые друзья! Поздравляю вас с 30-летием создания Психиатрической больницы №1. За прошедшие годы коллектив больницы прошёл большой, насыщенный путь, многое сделал для развития и совершенствования медицинского образования в нашей стране. За прошедшие годы коллектив больницы прошёл большой, насыщенный путь, многое сделал для развития и совершенствования медицинского образования в нашей стране. За прошедшие годы коллектив больницы прошёл большой, насыщенный путь, многое сделал для развития и совершенствования медицинского образования в нашей стране. За прошедшие годы коллектив больницы прошёл большой, насыщенный путь, многое сделал для развития и совершенствования медицинского образования в нашей стране. За прошедшие годы коллектив больницы прошёл большой, насыщенный путь, многое сделал для развития и совершенствования медицинского образования в нашей стране. За прошедшие годы коллек

C "Фондом помощи ЛГБТ" этот чекпойнт справляется. "Европейский суд по правам человека" модель повторять не стала, но по крайней мере не стала генерировать про другой суд и упомянула "международное гуманитарное сотрудничество". В тексте про психиатрическую больницу наблюдается зацикливание (стандартная проблема гпт), это можно будет попробовать исправить подбором параметров

Проверим теперь, не ухудшилось ли качество менее неожиданных для модели примеров

In [27]:
real_prompt = test['prompt'].loc[0]
prompt_meta = real_prompt.split(TITLE_TAG)[0] + TITLE_TAG
generate(prompt_meta)

'BOS PER ORG Геральдический совет, Геральдический совет TITLE Коллективу и ветеранам Геральдического совета TEXT Дорогие друзья! Поздравляю вас с 75-летием создания Геральдического совета. За прошедшие годы Геральдический совет вырос в крупное, авторитетное общественное объединение, которое объединяет представителей самых разных сословий, представителей разных поколений. Важно, что вы бережно храните традиции своих предшественников, стремитесь эффективно решать поставленные задачи, вносить значимый вклад в развитие отечественной культуры и искусства, в укрепление международного гуманитарного сотрудничества. И конечно, важно, что вы уделяете неустанное внимание реализации востребованных благотворительных, образовательных, просветительских инициатив, уделяете неустанное внимание реализации востребованных благотворительных, образовательных, просветительских инициатив, уделяете неустанное внимание реализации востребованных благотворительных, образовательных, просветительских инициатив, уде

In [28]:
real_prompt = test['prompt'].loc[1]
prompt_meta = real_prompt.split(TITLE_TAG)[0] + TITLE_TAG
generate(prompt_meta)

'BOS PER ORG Серовского исторического музея TITLE Коллективу Серовского исторического музея TEXT Дорогие друзья! Поздравляю вас с 75-летием Серовского исторического музея. История Серовского исторического музея тесно связана с именами выдающихся учёных, писателей, художников, которые внесли значимый вклад в развитие отечественной культуры, в укрепление её исторического, культурного, духовного потенциала. Важно, что вы бережно храните богатейшее собрание раритетов, которые по праву считаются бесценными памятниками отечественной культуры, которые по праву считаются бесценными достоянием нашего народа. И конечно, важно, что вы уделяете неустанное внимание развитию музейного дела, уделяете неустанное внимание популяризации музейных коллекций, их пополнению экспозициями, рассказывающими об истории родного края, его замечательных традициях и обычаях. Желаю вам успехов и всего наилучшего. EOS'

In [30]:
generate("BOS PER Марк Розовский ORG TITLE")

'BOS PER Марк Розовский ORG TITLE Марк Розовский, кинорежиссёр, сценарист, продюсер, сценарист, продюсер, сценарист, продюсер, сценарист, продюсер, сценарист, автор сценариев и программ для телевидения, радио и телевидения TEXT Уважаемый Марк Анатольевич! Примите поздравления с 80-летним юбилеем. Вы посвятили себя кинематографу, своим творчеством обогатили отечественный кинематограф, снискали заслуженное признание зрителей. И конечно, отмечу Ваш большой личный вклад в развитие отечественного телевидения, в популяризацию отечественного искусства. Желаю Вам здоровья, благополучия и всего наилучшего. EOS'

In [33]:
generate("BOS PER Анна Щербакова ORG TITLE")

'BOS PER Анна Щербакова ORG TITLE Анне Щербаковой, победительнице чемпионата мира по спортивной борьбе 2018 года в Будапеште в соревнованиях по греко-римской борьбе в весовой категории до 97 кг TEXT Уважаемая Анна Сергеевна! Поздравляю Вас с триумфом. Вы блестяще выступили на чемпионате мира в Будапеште, убедительно подтвердили свои лидерские позиции в мировом греко-римском спорте. И конечно, особые слова благодарности Вашим тренерам и наставникам, которые помогли Вам в напряжённой борьбе одолеть сильных соперников. Желаю Вам новых достижений и всего наилучшего. EOS'

In [34]:
generate("BOS PER Алина Загитова ORG TITLE")

'BOS PER Алина Загитова ORG TITLE Алине Загитовой, победительнице чемпионата мира по художественной гимнастике 2018 года в Будапеште в упражнениях с мячом TEXT Уважаемая Алина Сергеевна! Поздравляю Вас с триумфом. Вы блестяще выступили на чемпионате мира в Будапеште, убедительно подтвердили свои лидерские позиции в мировом фигурном катании. И конечно, особые слова признательности – в адрес Ваших тренеров, наставников, всех, кто поддерживал Вас на пути к успеху. Желаю Вам новых достижений и всего самого доброго. EOS'

Теперь модель чуть лучше помнит, чем именно занимаются люди: с профессией Розовского она справилась. Но опять наблюдается зацикливание.

Фигуристкам модель упорно приписывает другие виды спорта. Впрочем, модель угадала, что они весят меньше 97 кг

In [35]:
generate("BOS PER А. Петров ORG TITLE")

'BOS PER А. Петров ORG TITLE Участникам торжественного мероприятия, посвящённого 100-летию создания в России института скорой помощи имени А. П. Павлова TEXT Уважаемые друзья! Поздравляю вас с 100-летием создания в России института скорой помощи имени А. П. Павлова. За прошедшие годы институт прошёл большой, насыщенный путь, стал одним из признанных лидеров отечественной медицины. За прошедшие годы институт прошёл большой, насыщенный путь, стал одним из признанных лидеров отечественной медицины. За прошедшие годы институт прошёл большой, насыщенный путь, стал одним из признанных лидеров отечественной медицины. За прошедшие годы институт прошёл большой, насыщенный путь, стал одним из признанных лидеров отечественной медицины. За прошедшие годы институт прошёл большой, насыщенный путь, стал одним из признанных лидеров отечественной медицины. За прошедшие годы институт прошёл большой, насыщенный путь, стал одним из признанных лидеров отечественной медицины. Важно, что вы с уважением относ

In [36]:
generate("BOS PER А. Петрову ORG TITLE")

'BOS PER А. Петрову ORG TITLE А. Петрову, победителю чемпионата мира по водным видам спорта 2019 года в Будапеште в соревнованиях по плаванию на дистанции 200 метров брассом TEXT Уважаемый Александр Сергеевич! Поздравляю Вас с триумфом. Вы блестяще выступили на чемпионате мира в Будапеште, продемонстрировали отличную подготовку, отличную физическую подготовку и по праву стали первым в истории чемпионом мира в плавании брассом. И конечно, особые слова благодарности – Вашим тренерам, наставникам, всем, кто поддерживал Вас на пути к успеху. Желаю Вам новых достижений и всего самого доброго. EOS'

In [37]:
generate("BOS PER В. Иванов ORG TITLE")

'BOS PER В. Иванов ORG TITLE Коллективу и ветеранам АО «РЖД» TEXT Уважаемые друзья! Поздравляю вас с 25-летием создания акционерного общества «РЖД». За прошедшие годы «РЖД» накопил солидный опыт, заложил традиции надёжного, динамично развивающегося железнодорожного транспорта, стал одним из признанных лидеров отечественного железнодорожного транспорта. За прошедшие годы «РЖД» прошёл большой путь, заложил замечательные традиции, заложенные многими поколениями его предшественников. Сегодня «РЖД» – это динамично развивающийся, динамично развивающийся транспортный узел, способный обеспечить потребности самых разных регионов страны. Важно, что вы бережно храните традиции своих предшественников, стремитесь эффективно решать поставленные задачи, эффективно решать поставленные задачи на благо России и её граждан. Желаю вам успехов и всего наилучшего. EOS'

In [38]:
generate("BOS PER В. Иванову ORG TITLE")

'BOS PER В. Иванову ORG TITLE В. Иванову, победителю чемпионата мира по водным видам спорта 2019 года в Будапеште в соревнованиях по плаванию на дистанции 200 метров брассом TEXT Уважаемый Владимир Иванович! Поздравляю Вас с триумфом. Вы блестяще выступили на чемпионате мира в Будапеште, продемонстрировали отличную подготовку, отличную физическую подготовку, умение держать удар и уверенно идти вперёд. И конечно, особые слова благодарности Вашим тренерам и наставникам, которые помогли Вам в напряжённой борьбе с сильными соперниками, в напряжённой борьбе с сильными эмоциями и настроем на победу. Желаю Вам новых достижений и всего самого доброго. EOS'

Проблема с восприятием персон в именительном падеже сохраняется, но легко чинится

In [45]:
generate("BOS PER Бонч-Осмаловской ORG TITLE")

'BOS PER Бонч-Осмаловской ORG TITLE Коллективу и ветеранам АО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «НПО «'

In [47]:
generate("BOS PER Бонч-Осмаловской ORG TITLE Бонч-Осмаловской")

'BOS PER Бонч-Осмаловской ORG TITLE Бонч-Осмаловской городской клинической больнице TEXT Уважаемые друзья! Поздравляю вас с 75-летием создания больницы. За прошедшие годы больница прошла большой, насыщенный путь, стала одним из признанных лидеров в сфере здравоохранения. За прошедшие годы больница прошла большой, насыщенный путь, стала одним из признанных лидеров в сфере здравоохранения. За прошедшие годы больница прошла большой, насыщенный путь, стала одним из признанных лидеров в сфере здравоохранения. За прошедшие годы больница прошла большой, насыщенный путь, стала одним из признанных лидеров в сфере здравоохранения. За прошедшие годы больница прошла большой, насыщенный путь, стала одним из признанных лидеров в сфере здравоохранения. За прошедшие годы больница прошла большой, насыщенный путь, стала одним из признанных лидеров в сфере здравоохранения. Желаю вам успехов и всего самого доброго. EOS'

In [46]:
generate("BOS PER А. А. Бонч-Осмаловской ORG TITLE")

'BOS PER А. А. Бонч-Осмаловской ORG TITLE А. А. Бонч-Осмаловской, актрисе театра и кино, народной артистке РСФСР TEXT Уважаемая Александра Александровна! Примите поздравления с юбилейным Днём рождения. Вы посвятили себя актёрскому искусству, своим творчеством обогатили отечественный кинематограф, снискали заслуженное признание зрителей и зрителей разных поколений. И конечно, отмечу Ваш большой личный вклад в развитие отечественного кинематографа, в укрепление его авторитета как одного из признанных лидеров отечественного искусства. Желаю Вам здоровья, благополучия и всего наилучшего. EOS'

С нестандартными фамилиями также нужны инициалы

Посмотрим, что получится, если передавать текст, испорченный при автоматической постановке в дательный падеж

In [49]:
generate("BOS PER Иванову автору фильму ORG TITLE")

'BOS PER Иванову автору фильму ORG TITLE Иванову автору фильма «Иван Грозный, или Сага о царе Салтане» TEXT Уважаемый Иван Васильевич! Примите поздравления с 80-летним юбилеем. Вы по праву можете гордиться многими поколениями своих предшественников, которые внесли значимый вклад в развитие отечественного кинематографа, в развитие отечественного театра и кинематографа. И конечно, отмечу Ваш большой личный вклад в укрепление авторитета России как великой державы. Желаю Вам здоровья, благополучия и всего наилучшего. EOS'

In [50]:
generate("BOS PER Иванову основателю унивенситету ORG TITLE")

'BOS PER Иванову основателю унивенситету ORG TITLE Иванову основателю унивенситетского унивенситета TEXT Уважаемый Иван Иванович! Примите поздравления с 80-летним юбилеем. Вы посвятили себя служению Отечеству, своим трудом внесли значимый вклад в развитие отечественной школы физкультуры, в укрепление института физкультурника, в развитие массового спорта в нашей стране. И конечно, особо отмечу Ваш личный вклад в развитие отечественного спорта высших достижений, в укрепление института физкультурника в нашей стране. Желаю Вам здоровья, успехов и всего наилучшего. EOS'

In [51]:
generate("BOS PER Кардиганову герою Россию ORG TITLE")

'BOS PER Кардиганову герою Россию ORG TITLE Кардиганову герою России, актёру театра и кино, народному артисту России TEXT Уважаемый Кардиганович! Примите поздравления с Днём рождения. Вы посвятили свою жизнь служению искусству, своим талантом и благородством снискали высокое профессиональное признание и искреннюю любовь зрителей. И конечно, отмечу Вашу востребованную общественную деятельность, активное участие в реализации востребованных благотворительных, просветительских, патриотических инициатив. Желаю Вам здоровья, благополучия и всего наилучшего. EOS'

Модель умеет сама исправлять падеж, так что можно просто ставить все существительные в дательный падеж, даже если пользователь введет в поле какой-то текст помимо фамилии

Фамилию модель все-таки может портить, но первый раз она ее смогла повторить